# Prepare data to support DEM comparison

Various DEM/DSM and elevation values are used in the workflow. 
It's important to understand consistency between the DEM values.

DEMs to be compared

* GA 5m - LiDAR surveys between 2001 and 2015
* GA 1m - 2013 - "GDA94 / MGA zone 55" ("EPSG:28355")
* AGO 1m - Tamar 2008 - "GDA94 / MGA zone 55"
* AGO 7cm (DSM) - 2021 - "WGS 84 / UTM zone 55S" ("EPSG:32755")

Comparison methods to be considered

* Comparison to surveyed ground control points
* Comparison of downsampled minimum elevation
* Comparison of points over road



In [1]:
import geopandas as gpd
import pandas as pd
from pyproj import CRS

from matplotlib import pyplot as plt

## AGO data prep

7cm DSM

Projection is "WGS 84 / UTM zone 55S" or "EPSG:32755"

```
% cd ~/Documents/Projects/AGO/Elevation/Launceston/AGO/DEM_DSM_DTM/Source\ Elevation\ Data/AUS_Launceston_2021_DSM/dsm

# Make virtual raster
% gdalbuildvrt launceston-dsm-7cm.vrt *.tif

% gdalinfo launceston-dsm-7cm.vrt | grep "Size"      
Size is 168000, 204000
Pixel Size = (0.072000000000000,-0.072000000000000)

# Downsample
# original pixel size is 0.072
% gdalwarp -r average -tr 0.288 0.288 launceston-dsm-7cm.vrt launceston-dsm-average-28cm.tif


% gdalinfo launceston-dsm-average-28cm.tif|grep "Size"
Size is 42000, 51000
Pixel Size = (0.288000000000000,-0.288000000000000)
```



### Ground control points

Survey control masks for Tasmania: 
https://nre.tas.gov.au/land-tasmania/geospatial-infrastructure-surveying/geodetic-survey/survey-control-marks-database-(surcom)

In [2]:
# constrain using building dataset boundary
buildings_file = "/Users/Fangyuan/Library/CloudStorage/OneDrive-SharedLibraries-FrontierSI/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing/Data Exploration/overture/launceston_building.gpkg"

buildings  = gpd.read_file(buildings_file)

In [3]:
csv_file = "/Users/Fangyuan/FrontierSI/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing/Data Exploration/Ground control/Tasmania/SurcomBasicMkRep/survey_control.csv"


In [4]:
# Load CSV into a Pandas DataFrame
df = pd.read_csv(csv_file, encoding="ISO-8859-1")

# Convert into vector
# Use GDA2020 for better consistency with WGS84 and recent data

# Ensure column names match your CSV structure
easting_col = "GDA2020_E"
northing_col = "GDA2020_N"
zone_col = "MGA_ZONE"

# Function to generate CRS based on MGA Zone
def get_crs(mga_zone):
    epsg_code = 7800 + int(mga_zone)  # GDA2020 MGA Zone EPSG Codes (e.g., Zone 55 → EPSG:7855)
    return CRS.from_epsg(epsg_code)

# Convert to GeoDataFrame
df["geometry"] = gpd.points_from_xy(df[easting_col], df[northing_col])
gdf = gpd.GeoDataFrame(df, geometry="geometry")

# Assign the correct CRS based on the first row's MGA Zone
if zone_col in df.columns:
    mga_zone = df[zone_col].iloc[0]  # Assume all points share the same zone
    target_crs = get_crs(mga_zone)
    print("crs:", target_crs)
    gdf.set_crs(target_crs, inplace=True)


crs: EPSG:7855


In [5]:
# Inspect data
df.HGT_DATUM.unique()

array(['STATE', 'AHD83', nan, 'LOCAL', 'AHD79'], dtype=object)

Criteria used in DTM validation report

* MARKSTATUS ='EXISTING' and
* HOR_ORDER in ('1ST','2ND','3RD','4TH') and
* HGT_ORDER in (c) and
* HGT_DATUM ='AHD83'

In [6]:
# Filter use above rules
filter = (gdf.MARKSTATUS=='EXISTING') & (gdf.HGT_DATUM=='AHD83') 
filter = filter & (gdf.HOR_ORDER.isin(['1ST', '2ND', '3RD', '4TH']))
filter = filter & (gdf.HGT_ORDER.isin(['1ST','2ND','3RD', 'L3RD','L3RDLC']))
filter = filter & (gdf.GDA2020_PU<0.1)
gdf = gdf[filter]
gdf.head()

,PACK_ID,SCS_NAME,MGA_ZONE,GDA2020_E,GDA2020_N,GDA2020_PU,PU_METHOD,HEIGHT,HGT_DATUM,HGT_CLASS,HGT_ORDER,GDA94_E,GDA94_N,HOR_CLASS,HOR_ORDER,TARGET_STR,MARKSTATUS,DESCRIPT,ORDER_SYMB,geometry
5,ST780,WHITE ROCK LT,55,531973.177,5241746.121,0.016,Rigorous,21.045,AHD83,DIRECT,3RD,531972.763,5241744.711,B,2ND,LT,EXISTING,Navigation Light.,purpletriangle,POINT (531973.177 5241746.121)
6,HECBM2490,NaN,55,383830.022,5387081.611,0.019,Rigorous,396.182,AHD83,LEV3,L3RD,383829.586,5387080.143,B,2ND,NaN,EXISTING,"Brass HEC BM mushroom in concrete block, stone...",purpletriangle,POINT (383830.022 5387081.611)
8,SPM10307,NaN,55,407210.996,5451683.214,0.024,Rigorous,147.275,AHD83,GPS,3RD,407210.568,5451681.768,B,2ND,NaN,EXISTING,Bronze SPM disc in Telstra pit.,purpletriangle,POINT (407210.996 5451683.214)
11,SPM1121,NaN,55,449987.450,5435063.000,0.016,Rigorous,3.794,AHD83,LEV3,L3RD,449987.036,5435061.569,B,2ND,NaN,EXISTING,Brass pin in concrete block with CI cover box.,purpletriangle,POINT (449987.45 5435063)
12,DSM1508-03,NaN,55,515710.223,5408371.298,0.018,Rigorous,29.497,AHD83,GPS,3RD,515709.788,5408369.872,B,2ND,NaN,EXISTING,Brass Disc in Cover Box.,purpletriangle,POINT (515710.223 5408371.298)


In [7]:
# filter within building boundary

bounding = buildings.to_crs(target_crs).union_all().convex_hull
gcps = gdf[gdf.geometry.within(bounding)]

In [8]:
len(gcps)

468

In [9]:
gcps.HGT_CLASS.unique()

array(['LEV3', 'GPS', 'LEVU'], dtype=object)

In [10]:
# filter out unreliable, not at ground level
gcps = gcps[~gcps.DESCRIPT.isnull()]
gcps = gcps[~(gcps.DESCRIPT.str.lower().str.contains('unsuitable') | gcps.DESCRIPT.str.lower().str.contains('destroyed'))]
gcps = gcps[~(gcps.DESCRIPT.str.lower().str.contains('missing') | gcps.DESCRIPT.str.lower().str.contains('below ground'))]
gcps = gcps[~(gcps.DESCRIPT.str.lower().str.contains('sub-surface') | gcps.DESCRIPT.str.lower().str.contains('deep'))]
gcps = gcps[~(gcps.DESCRIPT.str.lower().str.contains('rod') | gcps.DESCRIPT.str.lower().str.contains('pole'))]
gcps = gcps[~(gcps.DESCRIPT.str.lower().str.contains('roof') | gcps.DESCRIPT.str.lower().str.contains('wall'))]

len(gcps)

184

In [11]:
gcps[gcps.DESCRIPT.str.lower().str.contains('brass pin')==True].DESCRIPT.unique()

array(['Brass pin in concrete block with coverbox over.',
       'Brass pin in coverbox', 'Brass pin in concrete with cover box.',
       'Brass pin in concrete with cover box. NOTE: This station was previously known as G143 and then 113/8.',
       'Brass pin in cover box.',
       'Brass pin in concrete block, coverbox over.',
       'Brass pin in concrete block, cover box over.',
       'Brass pin stamped 465 in concrete with steel cover box',
       'Brass pin in cover box in footpath.',
       'Brass pin in concrete, cover box over.', 'Brass pin in cover box',
       'Brass pin in coverbox.',
       'Brass pin in concrete block with coverbox over',
       'Brass pin in concrete with steel cover box.'], dtype=object)

In [12]:
len(gcps[gcps.DESCRIPT.str.lower().str.contains('brass pin')==True])

62

In [13]:
gcps.DESCRIPT.describe()

count                         184
unique                         63
top       Brass SPM disc in kerb.
freq                           29
Name: DESCRIPT, dtype: object

In [14]:
gcps.DESCRIPT.unique()

array(['Brass Lands disc in kerb.',
       'Brass pin in concrete block with coverbox over.',
       'Bronze SPM disc in kerb.', 'Pin in coverbox',
       'Brass pin in coverbox', 'Brass pin in concrete with cover box.',
       'Brass SPM disc in kerb.',
       'Brass pin in concrete with cover box. NOTE: This station was previously known as G143 and then 113/8.',
       'Brass pin in cover box.',
       'Brass pin in concrete block, coverbox over.',
       'Bronze disc in concrete block with coverbox over.',
       'Bronze SPM disc in pram crossing.',
       'Brass pin in concrete block, cover box over.',
       'Brass SPM disc in cover box.',
       'Brass Survey disc in concrete kerb.',
       'Brass mushroom in concrete, coverbox over.',
       'Brass Lands disc in Telstra pit.',
       'Brass pin stamped 465 in concrete with steel cover box',
       'Brass SPM Disc in cover box.', 'Brass mushroom in concrete.',
       'Brass SPM disc in concrete abutment.',
       'Brass SPM disc 

In [15]:
# Save as GeoJSON
gcps.to_file("/Users/Fangyuan/FrontierSI/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing/Data Exploration/Ground control/Tasmania/tasmania_survey_control_filtered.geojson", driver="GeoJSON")

In [16]:
# Save as GeoJSON
gcps[gcps.DESCRIPT.str.lower().str.contains('brass pin')==True].to_file("/Users/Fangyuan/FrontierSI/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing/Data Exploration/Ground control/Tasmania/tasmania_survey_control_filtered_brass_pin.geojson", driver="GeoJSON")